In [1]:
import pandas as pd
# import numpy as np
import matplotlib.pyplot as plt

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.metrics import classification_report

In [2]:
incidents = pd.read_csv("../incident_data/yearly_incident_data/all_incidents.csv")
incidents = incidents.dropna()
incidents.head()

C:\Users\hasse\anaconda3\envs\PythonAdv\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,publicAddress,caseNumber,precinct,offense,description,ucrCode,lat,lon,neighborhood,incidentDate,incidentTime
0,0056Xx Lyndale Av S,MP2010000051,5,CSCR,Crim Sex Cond-Rape,3.0,44.900291,-93.288239,Windom,2010/01/01,00:30
1,0001Xx 62 St W,MP2010000099,5,ROBPAG,Robbery Per Agg,4.0,44.890636,-93.280041,Windom,2010/01/01,01:05
2,0039Xx 5 Av S,MP2010000373,3,BURGD,Burglary Of Dwelling,6.0,44.931448,-93.268841,Bryant,2010/01/01,04:18
3,0013Xx Nicollet Av S,MP2010000305,1,ASLT2,Asslt W/Dngrs Weapon,5.0,44.970506,-93.277714,Loring Park,2010/01/01,03:00
4,0034Xx Oliver Av N,MP2010000229,4,BURGD,Burglary Of Dwelling,6.0,45.017746,-93.306988,Folwell,2010/01/01,02:23


In [3]:
incidents['incidentDate'] = pd.to_datetime(incidents['incidentDate'], format='%Y/%m/%d')
incidents['month']= incidents['incidentDate'].dt.month 
incidents['weekday'] =  incidents['incidentDate'].dt.dayofweek
incidents['day'] =  incidents['incidentDate'].dt.day
incidents.head()

,publicAddress,caseNumber,precinct,offense,description,ucrCode,lat,lon,neighborhood,incidentDate,incidentTime,month,weekday,day
0,0056Xx Lyndale Av S,MP2010000051,5,CSCR,Crim Sex Cond-Rape,3.0,44.900291,-93.288239,Windom,2010-01-01,00:30,1,4,1
1,0001Xx 62 St W,MP2010000099,5,ROBPAG,Robbery Per Agg,4.0,44.890636,-93.280041,Windom,2010-01-01,01:05,1,4,1
2,0039Xx 5 Av S,MP2010000373,3,BURGD,Burglary Of Dwelling,6.0,44.931448,-93.268841,Bryant,2010-01-01,04:18,1,4,1
3,0013Xx Nicollet Av S,MP2010000305,1,ASLT2,Asslt W/Dngrs Weapon,5.0,44.970506,-93.277714,Loring Park,2010-01-01,03:00,1,4,1
4,0034Xx Oliver Av N,MP2010000229,4,BURGD,Burglary Of Dwelling,6.0,45.017746,-93.306988,Folwell,2010-01-01,02:23,1,4,1


In [4]:
one_hot = pd.get_dummies(incidents['neighborhood'])
incidents = incidents.drop('neighborhood', axis=1)
incidents =incidents.join(one_hot)
incidents.head()

,publicAddress,caseNumber,precinct,offense,description,ucrCode,lat,lon,incidentDate,incidentTime,...,Ventura Village,Victory,Waite Park,Webber - Camden,Wenonah,West Calhoun,Whittier,Willard - Hay,Windom,Windom Park
0,0056Xx Lyndale Av S,MP2010000051,5,CSCR,Crim Sex Cond-Rape,3.0,44.900291,-93.288239,2010-01-01,00:30,...,0,0,0,0,0,0,0,0,1,0
1,0001Xx 62 St W,MP2010000099,5,ROBPAG,Robbery Per Agg,4.0,44.890636,-93.280041,2010-01-01,01:05,...,0,0,0,0,0,0,0,0,1,0
2,0039Xx 5 Av S,MP2010000373,3,BURGD,Burglary Of Dwelling,6.0,44.931448,-93.268841,2010-01-01,04:18,...,0,0,0,0,0,0,0,0,0,0
3,0013Xx Nicollet Av S,MP2010000305,1,ASLT2,Asslt W/Dngrs Weapon,5.0,44.970506,-93.277714,2010-01-01,03:00,...,0,0,0,0,0,0,0,0,0,0
4,0034Xx Oliver Av N,MP2010000229,4,BURGD,Burglary Of Dwelling,6.0,45.017746,-93.306988,2010-01-01,02:23,...,0,0,0,0,0,0,0,0,0,0


In [11]:
selected_features = incidents.drop(['publicAddress', 'caseNumber', 'offense', 'description', 'lat', 'lon', 'incidentDate', 'incidentTime', 'ucrCode'], axis=1)

selected_features.head()

,precinct,month,weekday,day,Armatage,Audubon Park,Bancroft,Beltrami,Bottineau,Bryant,...,Ventura Village,Victory,Waite Park,Webber - Camden,Wenonah,West Calhoun,Whittier,Willard - Hay,Windom,Windom Park
0,5,1,4,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,5,1,4,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,3,1,4,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,1,4,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,1,4,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
X = selected_features
y = incidents['ucrCode']

In [ ]:
# Start of Naive Bayes work

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
nb = MultinomialNB()
nb.fit(X_train, y_train)

MultinomialNB()

In [15]:
predictions = nb.predict(X_test)
print(predictions)

[7. 7. 7. ... 7. 7. 7.]


In [16]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

0.5471266390917892